In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from catboost import CatBoostClassifier
import xgboost as xgb
import random

In [4]:
data = pd.read_csv("BTC-USD_SIGNALS.csv")

In [5]:
data

,Date,Open,High,Low,Close,Volume,Variation,RSI,MACD,MACD_H,Confirmation Time,Transactions,Miners Revenue,FnG
0,2018-02-01,10237.299805,10288.799805,8812.280273,9170.540039,-0.292146,-2.712213,-1.043523,-1.108359,-0.475907,1.144750,-0.774692,-0.186892,-0.583209
1,2018-02-02,9142.280273,9142.280273,7796.490234,8830.750000,-0.152676,-1.011481,-1.204172,-1.202571,-0.654534,1.144750,-0.774692,-0.186892,-1.244407
2,2018-02-03,8852.120117,9430.750000,8251.629883,9174.910156,-0.427993,0.955604,-1.632900,-1.235294,-0.618233,1.114249,-0.774692,-0.186892,-0.142410
3,2018-02-04,9175.700195,9334.870117,8031.220215,8277.009766,-0.437581,-2.584947,-1.626830,-1.320379,-0.741794,1.114249,-1.144752,-0.264496,-0.847688
4,2018-02-05,8270.540039,8364.839844,6756.680176,6955.270020,-0.326118,-4.184569,-1.814295,-1.479953,-1.057732,1.114249,-1.144752,-0.264496,-1.420727
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1679,2022-09-04,19832.470703,19999.689453,19636.816406,19986.712891,0.478227,0.148948,-1.113287,-0.844382,-0.254845,-1.063067,-0.648298,-0.307825,-1.024008
1680,2022-09-05,19988.789063,20031.160156,19673.046875,19812.371094,0.658018,-0.278482,-1.133325,-0.831832,-0.166540,-0.742360,-0.648298,-0.307825,-0.891768
1681,2022-09-06,19817.724609,20155.269531,18800.171875,18837.667969,1.393318,-1.325705,-1.548973,-0.892284,-0.308652,-0.742360,-0.165203,-0.359746,-0.935848
1682,2022-09-07,18837.683594,19427.171875,18644.466797,19290.324219,0.981876,0.568952,-1.180050,-0.893278,-0.249058,-0.742360,-0.165203,-0.359746,-0.847688


In [6]:
predict_data = data.copy().drop(["Open","Close","High","Low"],axis=1)
max_days = 5
target_range = 2
for i in range(1,max_days):#2jours
    predict_data[["Variation-{}".format(i),"Vol-{}".format(i),"RSI-{}".format(i),"MACD-{}".format(i),"MACD_H-{}".format(i),"CONF-{}".format(i),"TRANS-{}".format(i),"REV-{}".format(i),"FnG-{}".format(i)]] = data[["Variation","Volume","RSI","MACD","MACD_H","Confirmation Time","Transactions","Miners Revenue","FnG"]].shift(i)
    #predict_data[["Variation-{}".format(i),"Vol-{}".format(i),"RSI-{}".format(i),"MACD-{}".format(i),"MACD_H-{}".format(i)]] = data[["Variation","Volume","RSI","MACD","MACD_H"]].shift(i)
#predict_data["Target"] = (data["Variation"].shift(-1) >= 0)
predict_data["Target"] = (data["Close"].shift(-target_range) - data["Close"] >= 0)
predict_data["Target"] = np.where(predict_data["Target"] == True, 1, 0)
predict_data.dropna(inplace=True)
predict_data.reset_index(inplace=True,drop=True)
predict_data = predict_data[0:len(predict_data)-target_range]

In [7]:
predict_data = predict_data[[i % int(max_days / 3) == 0 for i in range(len(predict_data))]]

In [8]:
predict_data

,Date,Volume,Variation,RSI,MACD,MACD_H,Confirmation Time,Transactions,Miners Revenue,FnG,Variation-1,Vol-1,RSI-1,MACD-1,MACD_H-1,CONF-1,TRANS-1,REV-1,FnG-1,Variation-2,Vol-2,RSI-2,MACD-2,MACD_H-2,CONF-2,TRANS-2,REV-2,FnG-2,Variation-3,Vol-3,RSI-3,MACD-3,MACD_H-3,CONF-3,TRANS-3,REV-3,FnG-3,Variation-4,Vol-4,RSI-4,MACD-4,MACD_H-4,CONF-4,TRANS-4,REV-4,FnG-4,Target
0,2018-02-05,-0.326118,-4.184569,-1.814295,-1.479953,-1.057732,1.114249,-1.144752,-0.264496,-1.420727,-2.584947,-0.437581,-1.626830,-1.320379,-0.741794,1.114249,-1.144752,-0.264496,-0.847688,0.955604,-0.427993,-1.632900,-1.235294,-0.618233,1.114249,-0.774692,-0.186892,-0.142410,-1.011481,-0.152676,-1.204172,-1.202571,-0.654534,1.144750,-0.774692,-0.186892,-1.244407,-2.712213,-0.292146,-1.043523,-1.108359,-0.475907,1.144750,-0.774692,-0.186892,-0.583209,1
1,2018-02-06,-0.088527,2.918547,-1.212491,-1.523793,-0.973190,0.453384,-1.144752,-0.264496,-1.552966,-4.184569,-0.326118,-1.814295,-1.479953,-1.057732,1.114249,-1.144752,-0.264496,-1.420727,-2.584947,-0.437581,-1.626830,-1.320379,-0.741794,1.114249,-1.144752,-0.264496,-0.847688,0.955604,-0.427993,-1.632900,-1.235294,-0.618233,1.114249,-0.774692,-0.186892,-0.142410,-1.011481,-0.152676,-1.204172,-1.202571,-0.654534,1.144750,-0.774692,-0.186892,-1.244407,1
2,2018-02-07,-0.331965,-0.495589,-1.457407,-1.552132,-0.860381,0.453384,-1.959889,-0.596734,-0.318730,2.918547,-0.088527,-1.212491,-1.523793,-0.973190,0.453384,-1.144752,-0.264496,-1.552966,-4.184569,-0.326118,-1.814295,-1.479953,-1.057732,1.114249,-1.144752,-0.264496,-1.420727,-2.584947,-0.437581,-1.626830,-1.320379,-0.741794,1.114249,-1.144752,-0.264496,-0.847688,0.955604,-0.427993,-1.632900,-1.235294,-0.618233,1.114249,-0.774692,-0.186892,-0.142410,1
3,2018-02-08,-0.323021,2.134561,-1.063630,-1.504877,-0.549827,0.453384,-1.959889,-0.596734,-0.583209,-0.495589,-0.331965,-1.457407,-1.552132,-0.860381,0.453384,-1.959889,-0.596734,-0.318730,2.918547,-0.088527,-1.212491,-1.523793,-0.973190,0.453384,-1.144752,-0.264496,-1.552966,-4.184569,-0.326118,-1.814295,-1.479953,-1.057732,1.114249,-1.144752,-0.264496,-1.420727,-2.584947,-0.437581,-1.626830,-1.320379,-0.741794,1.114249,-1.144752,-0.264496,-0.847688,1
4,2018-02-09,-0.452131,1.422827,-0.830140,-1.412981,-0.171285,0.207605,-1.959889,-0.596734,0.033910,2.134561,-0.323021,-1.063630,-1.504877,-0.549827,0.453384,-1.959889,-0.596734,-0.583209,-0.495589,-0.331965,-1.457407,-1.552132,-0.860381,0.453384,-1.959889,-0.596734,-0.318730,2.918547,-0.088527,-1.212491,-1.523793,-0.973190,0.453384,-1.144752,-0.264496,-1.552966,-4.184569,-0.326118,-1.814295,-1.479953,-1.057732,1.114249,-1.144752,-0.264496,-1.420727,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1673,2022-09-02,0.673668,-0.255088,-0.707947,-0.867803,-0.506193,-1.063067,-0.310919,-0.277036,-0.803608,0.047069,0.726988,-1.336325,-0.871305,-0.646449,-1.313931,-0.310919,-0.277036,-1.024008,0.277823,0.880353,-1.406081,-0.880154,-0.841249,-1.313931,-0.310919,-0.277036,-0.891768,-0.691652,0.943757,-1.653641,-0.871357,-1.020465,-1.313931,-1.414496,-0.174982,-0.715449,0.845674,0.850751,-1.597787,-0.821959,-1.096574,-1.263537,-1.414496,-0.174982,-0.847688,1
1674,2022-09-03,0.395940,-0.231177,-0.964656,-0.866931,-0.401653,-1.063067,-0.648298,-0.307825,-0.979928,-0.255088,0.673668,-0.707947,-0.867803,-0.506193,-1.063067,-0.310919,-0.277036,-0.803608,0.047069,0.726988,-1.336325,-0.871305,-0.646449,-1.313931,-0.310919,-0.277036,-1.024008,0.277823,0.880353,-1.406081,-0.880154,-0.841249,-1.313931,-0.310919,-0.277036,-0.891768,-0.691652,0.943757,-1.653641,-0.871357,-1.020465,-1.313931,-1.414496,-0.174982,-0.715449,0
1675,2022-09-04,0.478227,0.148948,-1.113287,-0.844382,-0.254845,-1.063067,-0.648298,-0.307825,-1.024008,-0.231177,0.395940,-0.964656,-0.866931,-0.401653,-1.063067,-0.648298,-0.307825,-0.979928,-0.255088,0.673668,-0.707947,-0.867803,-0

In [9]:
#corr = predict_data.corr("pearson")
#corr[["Target"]].to_clipboard()
#corr[["Target"]]

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
split_mode = "RAND"

if split_mode == "RAND":
    train_data, test_data, train_labels, test_labels = train_test_split(predict_data.drop(["Date","Target"],axis=1), predict_data["Target"], test_size=0.33, random_state=100)
elif split_mode == "STATIC":
    train_data = predict_data[0:int(0.66*len(predict_data))].drop(["Date","Target"],axis=1)
    train_labels = predict_data[0:int(0.66*len(predict_data))]["Target"]
    test_data = predict_data[int(0.66*len(predict_data)):len(predict_data)].drop(["Date","Target"],axis=1)
    test_labels = predict_data[int(0.66*len(predict_data)):len(predict_data)]["Target"]

In [12]:
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

In [13]:
algo = "MLP"

if algo == "MLP":
    model = MLPClassifier(hidden_layer_sizes=(200,2000,200),verbose=True)
elif algo == "RF":
    model = RandomForestClassifier(n_estimators=100,verbose=False)
elif algo == "CAT":
    model = CatBoostClassifier(iterations=100,depth=12)
elif algo == "TREE":
    model = DecisionTreeClassifier(splitter="random")
model.fit(train_data,train_labels)

Iteration 1, loss = 0.74636336
Iteration 2, loss = 0.69649579
Iteration 3, loss = 0.67046350
Iteration 4, loss = 0.66180363
Iteration 5, loss = 0.65379488
Iteration 6, loss = 0.63297506
Iteration 7, loss = 0.61386179
Iteration 8, loss = 0.58982201
Iteration 9, loss = 0.56679113
Iteration 10, loss = 0.53945868
Iteration 11, loss = 0.51540729
Iteration 12, loss = 0.50061452
Iteration 13, loss = 0.46480858
Iteration 14, loss = 0.43590603
Iteration 15, loss = 0.39575270
Iteration 16, loss = 0.37467826
Iteration 17, loss = 0.33360911
Iteration 18, loss = 0.30891724
Iteration 19, loss = 0.28295675
Iteration 20, loss = 0.24615849
Iteration 21, loss = 0.22562006
Iteration 22, loss = 0.21413693
Iteration 23, loss = 0.18695444
Iteration 24, loss = 0.18091632
Iteration 25, loss = 0.15884428
Iteration 26, loss = 0.14444562
Iteration 27, loss = 0.11347248
Iteration 28, loss = 0.09183450
Iteration 29, loss = 0.06705855
Iteration 30, loss = 0.05440977
Iteration 31, loss = 0.04569979
Iteration 32, los

MLPClassifier(hidden_layer_sizes=(200, 2000, 200), verbose=True)

In [14]:
model.score(test_data,test_labels)

0.6046931407942239

In [15]:
model.score(train_data,train_labels)

0.9973309608540926

In [16]:
from sklearn.metrics import confusion_matrix

In [17]:
preds = model.predict(test_data)

In [18]:
conf = confusion_matrix(test_labels,preds)
conf

array([[145, 116],
       [103, 190]], dtype=int64)

In [19]:
TN = conf[0,0] / (conf[0,0] + conf[1,0])
TN

0.5846774193548387

In [20]:
TP = conf[1,1] / (conf[1,1] + conf[0,1])
TP

0.6209150326797386

## Cross Validation

In [24]:
def cross_validation_loop(predict_data):
    scores = []
    TPs = []
    TNs = []
    for randomize in range(1,20):
        train_data, test_data, train_labels, test_labels = train_test_split(predict_data.drop(["Date","Target"],axis=1), predict_data["Target"], test_size=0.33, random_state=randomize)
        model = MLPClassifier(hidden_layer_sizes=(200,2000,200))
        model.fit(train_data, train_labels)
        scores.append(model.score(test_data, test_labels))
        conf = confusion_matrix(test_labels,model.predict(test_data))
        TN = conf[0,0] / (conf[0,0] + conf[1,0])
        TP = conf[1,1] / (conf[1,1] + conf[0,1])
        TPs.append(TP)
        TNs.append(TN)
        print("Score : {} || TP : {} || TN : {}".format(scores[-1],TP,TN))


    return scores,TPs,TNs

In [25]:
scores,TPs,TNs = cross_validation_loop(predict_data)

Score : 0.5848375451263538 || TP : 0.6115107913669064 || TN : 0.5579710144927537
Score : 0.5649819494584838 || TP : 0.5966666666666667 || TN : 0.5275590551181102
Score : 0.5794223826714802 || TP : 0.5935483870967742 || TN : 0.5614754098360656
Score : 0.5848375451263538 || TP : 0.6140350877192983 || TN : 0.5539033457249071
Score : 0.5794223826714802 || TP : 0.603125 || TN : 0.5470085470085471
Score : 0.592057761732852 || TP : 0.6084142394822006 || TN : 0.5714285714285714
Score : 0.5866425992779783 || TP : 0.6142857142857143 || TN : 0.5583941605839416
Score : 0.6101083032490975 || TP : 0.6466666666666666 || TN : 0.5669291338582677
Score : 0.572202166064982 || TP : 0.6214285714285714 || TN : 0.5218978102189781
Score : 0.6101083032490975 || TP : 0.5973597359735974 || TN : 0.6254980079681275
Score : 0.6101083032490975 || TP : 0.6351791530944625 || TN : 0.5789473684210527
Score : 0.555956678700361 || TP : 0.5471014492753623 || TN : 0.564748201438849
Score : 0.5902527075812274 || TP : 0.58823

In [26]:
from statistics import mean


print("score moyen : {} || TP moyen : {} || TN moyen : {}".format(mean(scores),mean(TPs),mean(TNs)))

score moyen : 0.5842675280258408 || TP moyen : 0.6065943975464273 || TN moyen : 0.559070903929586
